In [1]:
# Для работы с большими датасетами используем библиотеку cudf от NVIDIA, с целью перенаправления вычислений на GPU

import cudf as pd
# import cupy as cp
import numpy as np
# import matplotlib.pyplot as plt
# import pandas

In [2]:
# Загружаем исходные данные

adv_data = pd.read_csv('data/adv_data.csv')
users_data = pd.read_csv('data/users_data.csv')
event_data = pd.read_csv('data/event_data.csv')

In [3]:
adv_data.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,<NA>
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,<NA>
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,<NA>
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,<NA>
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [4]:
event_data.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,+79140092454,show,2023-10-05 12:03:04.125806,<NA>,<NA>,<NA>,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,+79125904946,show,2023-05-16 10:16:54.554523,<NA>,<NA>,<NA>,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,+79184556300,show,2023-12-23 22:46:44.066599,<NA>,<NA>,<NA>,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,+79119287528,show,2023-07-30 05:25:46.285514,<NA>,<NA>,<NA>,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,+79119287528,click,2023-07-30 05:25:46.285514,<NA>,2023-07-30 05:42:59.014970,2.18,USD,loan


In [5]:
users_data.head()

,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,<NA>
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,<NA>
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,<NA>
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,<NA>
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,<NA>


In [6]:
# event_data.loc[event_data['user_phone'].str.contains('79135447910')]

In [7]:
# adv_data.loc[adv_data['user_id'].str.contains('9501e80e595c809f00d47404fa6eea15dac1de44473892')]

In [8]:
# adv_data['campaign_type'].unique()

In [9]:
# Переменная, в которой хранится значение курса доллара США на текущий день, для приведения колонок campaign_cost и payout к общему рублёвому значению

usdrub = 90.41

In [10]:
# Метод, приводящий значения в долларах США к рублевым значениям в столбце payout

def convert_to_rub(row):
    x = row['payout']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [11]:
# Метод, приводящий значения в долларах США к рублевым значениям в столбце campaign_cost

def convert_campaign_cost_to_rub(row):
    x = row['campaign_cost']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [12]:
# Применение вышеуказанных методов

adv_data['campaign_cost'] = adv_data.apply(convert_campaign_cost_to_rub, axis=1)

In [13]:
# Удаление столбцов currency из датасетов event_data, adv_data

adv_data.drop('currency', inplace=True, axis=1)

In [14]:
adv_data['campaign_cost'].sum()

422015060.63579994

In [15]:
# Экспериментально установили, что неуникальные значения пользовательских данных - это значения, в которых в ячейке 'user_id' записано значение NaN

users_data = users_data[~users_data['user_id'].isna()]

In [16]:
# Заполнили значения NaN в столбце campaign_cost нулями

adv_data['campaign_cost'].fillna(0.0, inplace=True)

In [17]:
# Экспериментально установили, что столбец delete_dtm полностью пустой, поэтому удаляю его

users_data.drop('delete_dtm', axis=1, inplace=True)

In [18]:
# Из-за того, что номера телефонов в таблицах event_data, users_data записаны по-разному (со знаком + и без него) в следующих трех ячейках приводим телефоны к единому написанию:

# 1) создаем numpy-массивы с нулевыми значениями, по размеру идентичные размеру столбцов user_phone в вышеуказанных датасетах

user_phones = users_data['user_phone'].to_numpy()
user_test = np.zeros_like(user_phones)

In [19]:
# 2) заполняем массивы строковыми значениями номеров телефонов без знаков +

for i, val in enumerate(user_phones):
    user_test[i] = str(val).strip('+')

In [20]:
# 3) заменяем исходные значения в датасетах заполненными массивами


users_data['user_phone'] = user_test

In [21]:
# Для экономии памяти удаляем ненужные массивы

del user_test

In [22]:
# Создаем массив уникальных значений ID пользователей

users_uuids = users_data['user_id'].unique()

In [23]:
# Из датасета adv_data выбираем значения, которым соответствуют значения пользователей

adv_data = adv_data[adv_data['user_id'].isin(users_uuids)]

In [24]:
adv_data['campaign_cost'].sum()

238736357.63579997

In [25]:
# Проверяем оставшиеся после этого значения в таблице adv_data

# adv_data.info()

In [26]:
# Создаем массив уникальных значений телефонов пользователей

user_phones = users_data['user_phone'].unique()

In [27]:
# users_data.info()

In [28]:
# event_data.info()

In [29]:
# проверяем оставшиеся после этого значения в таблице adv_data

# event_data.info()

In [30]:
# Соединяем таблицы adv_data, users_data по ID пользователей аналогично SQL-методу left-join (остаются все значения из датасета adv_data и соответствующие им значения из 
# датасета users_data)

adv_data_plus_user_data = adv_data.merge(users_data, how='left', on='user_id')

In [31]:
# Проверяем новый датасет

# adv_data_plus_user_data.info()

In [32]:
# Для экономии памяти удаляем ненужные массивы и таблицы

del users_uuids, adv_data, users_data

In [33]:
# adv_data_plus_user_data.loc[adv_data_plus_user_data['user_phone'] == '79101867847']

In [34]:
# event_data.loc[event_data['user_phone'] == '79101867847']

In [35]:
adv_data_plus_user_data_with_dummies = pd.get_dummies(adv_data_plus_user_data, columns=['campaign_type', 'source_medium', 'interface'], dtype='int')[['user_phone', 'campaign_cost', 
       'campaign_type_click', 'campaign_type_conversion', 'campaign_type_show',
       'source_medium_android / mobile_app', 'source_medium_bing / organic',
       'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
       'source_medium_google / organic', 'source_medium_ios / mobile_app',
       'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
       'source_medium_regular / email', 'source_medium_regular / sms',
       'source_medium_ru.android.bankproduct / mobile_app',
       'source_medium_ru.android.bankproduct_rustore / mobile_app',
       'source_medium_trigger / sms', 'source_medium_vk / cpc',
       'source_medium_web.telegram.org / referral ',
       'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
       'source_medium_yandex / cpc', 'source_medium_yandex / organic',
       'source_medium_zen.yandex.ru / cpm', 'interface_api', 'interface_app',
       'interface_web']]

In [36]:
adv_data_plus_user_data_with_dummies.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 9999908 entries, 0 to 9999907
Data columns (total 28 columns):
 #   Column                                                     Dtype
---  ------                                                     -----
 0   user_phone                                                 object
 1   campaign_cost                                              float64
 2   campaign_type_click                                        int64
 3   campaign_type_conversion                                   int64
 4   campaign_type_show                                         int64
 5   source_medium_android / mobile_app                         int64
 6   source_medium_bing / organic                               int64
 7   source_medium_email / email                                int64
 8   source_medium_finance.rambler.ru / cpm                     int64
 9   source_medium_google / organic                             int64
 10  source_medium_ios / mobile_app       

In [37]:
del adv_data_plus_user_data

In [38]:
adv_data_plus_user_data_with_dummies = adv_data_plus_user_data_with_dummies.sort_values(by='user_phone')

In [39]:
adv_data_plus_user_data_with_dummies_1 = adv_data_plus_user_data_with_dummies.iloc[0:2500000].groupby('user_phone').sum()

In [40]:
adv_data_plus_user_data_with_dummies_2 = adv_data_plus_user_data_with_dummies.iloc[2500000:5000000].groupby('user_phone').sum()

In [41]:
adv_data_plus_user_data_with_dummies_3 = adv_data_plus_user_data_with_dummies.iloc[5000000:7500000].groupby('user_phone').sum()

In [42]:
adv_data_plus_user_data_with_dummies_4 = adv_data_plus_user_data_with_dummies.iloc[7500000:].groupby('user_phone').sum()

In [43]:
adv_data_plus_user_data_with_dummies_all = pd.concat([
    adv_data_plus_user_data_with_dummies_1,
    adv_data_plus_user_data_with_dummies_2
])

In [44]:
del adv_data_plus_user_data_with_dummies_1, adv_data_plus_user_data_with_dummies_2

In [45]:
adv_data_plus_user_data_with_dummies_all_1 = pd.concat([
    adv_data_plus_user_data_with_dummies_3,
    adv_data_plus_user_data_with_dummies_4
])

In [46]:
del adv_data_plus_user_data_with_dummies_3, adv_data_plus_user_data_with_dummies_4

In [47]:
adv_data_plus_user_data_with_dummies_all_final = pd.concat([
    adv_data_plus_user_data_with_dummies_all,
    adv_data_plus_user_data_with_dummies_all_1
])

In [48]:
del adv_data_plus_user_data_with_dummies_all, adv_data_plus_user_data_with_dummies_all_1

In [49]:
adv_data_plus_user_data_with_dummies_all_final.to_csv('data/adv_data_plus_users_groupped.csv')

In [50]:
del adv_data_plus_user_data_with_dummies_all_final

In [51]:
event_data = pd.read_csv('data/event_data.csv')

In [52]:
event_data.loc[event_data['user_phone'].str.contains('79135447910')]

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
4667231,9501e80e595c809f00d47404fa6eea15dac1de44473892...,+79135447910,show,2023-09-01 19:39:27.225870,<NA>,<NA>,<NA>,USD,deposits


In [53]:
event_data['payout'] = event_data.apply(convert_to_rub, axis=1)
event_data.drop('currency', inplace=True, axis=1)
event_data['payout'].fillna(0.0, inplace=True)



In [54]:
event_phones = event_data['user_phone'].to_numpy()
np_test = np.zeros_like(event_phones)

for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')

event_data['user_phone'] = np_test

del np_test

event_data = event_data[event_data['user_phone'].isin(user_phones)]

In [55]:
event_data['payout'].sum()

298360421.86520004

In [56]:
event_data.drop(['id', 'event_dtm', 'update_dtm'], inplace=True, axis=1)

In [57]:
event_data.head(1)

,user_phone,event_type,event_status,payout,product_type
0,79140092454,show,<NA>,0.0,mortgage


In [58]:
event_data_with_dummies = pd.get_dummies(event_data, columns=['event_type', 'product_type', 'event_status'])

In [59]:
del event_data, user_phones

In [60]:
event_data_with_dummies_groupped = event_data_with_dummies.groupby('user_phone').sum()

In [ ]:
del event_data_with_dummies

In [62]:
event_data_with_dummies_groupped.to_csv('data/event_data_with_dummies_groupped.csv')

In [63]:
del event_data_with_dummies_groupped

In [2]:
event_data_with_dummies_groupped = pd.read_csv('data/event_data_with_dummies_groupped.csv')

In [3]:
adv_data_plus_user_data_with_dummies_groupped = pd.read_csv('data/adv_data_plus_users_groupped.csv')

In [4]:
all_data = adv_data_plus_user_data_with_dummies_groupped.merge(event_data_with_dummies_groupped, how='inner', on='user_phone')

In [5]:
del event_data_with_dummies_groupped, adv_data_plus_user_data_with_dummies_groupped

In [6]:
all_data.to_csv('data/all_data_groupped.csv')

In [7]:
del all_data